**Table of contents**<a id='toc0_'></a>    
1. [生成器](#toc1_)    
1.1. [目的](#toc1_1_)    
1.2. [使用"迭代器"完成部分功能](#toc1_2_)    
1.2.1. [尝试1](#toc1_2_1_)    
1.2.2. [尝试2](#toc1_2_2_)    
1.3. [生成器](#toc1_3_)    
1.3.1. [什么是生成器](#toc1_3_1_)    
1.3.2. [生成器与迭代器](#toc1_3_2_)    
1.3.3. [创建生成器方法1](#toc1_3_3_)    
1.3.4. [创建生成器方法2](#toc1_3_4_)    
1.3.5. [yield关键字](#toc1_3_5_)    
1.3.6. [获取生成器def代码块中return值](#toc1_3_6_)    
1.4. [使用send唤醒](#toc1_4_)    
1.4.1. [测试send](#toc1_4_1_)    
1.4.2. [注意](#toc1_4_2_)    
1.5. [总结](#toc1_5_)    
1.6. [作业](#toc1_6_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=true
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# 1. <a id='toc1_'></a>[生成器](#toc0_)

## 1.1. <a id='toc1_1_'></a>[目的](#toc0_)
如下图，是一个初中阶段的二元一次方程，其中Y=2X+1

![](../image/03/generator01.png)

- 当X的值是0，那么Y的值是1

- 当X的值是1，那么Y的值是3

- 当X的值是2，那么Y的值是5

...以此类推...

现有如下需求，请问该怎样实现呢？

>1. 不确定个数的 点的坐标
>2. 第1次点的坐标X值是0，得到的Y值1；将此次的Y当做做第2次点的X坐标值
>3. 即 第2次点的坐标值是1，得到Y值3；将此次的Y当做做第3次点的X坐标值
>4. 即 第3次点的坐标值是3，得到Y值7；将此次的Y当做做第4次点的X坐标值</br>
>​   **...以此类推...**
>5. 在不确定第X次时，可能需要修改方程(Y=2X+1)组中的2与1的值

想想该怎样实现呢？

就用到我们今天要研究的知识点：生成器

## 1.2. <a id='toc1_2_'></a>[使用"迭代器"完成部分功能](#toc0_)

### 1.2.1. <a id='toc1_2_1_'></a>[尝试1](#toc0_)
上面的问题中，假如需要5个，实现较为简单，参考代码如下：

In [1]:
point_x_y_list = []  # 定义一个列表用来存储元组
x = 0
for _ in range(5):
    y = 2 * x + 1  # 根据x的值计算出y的值
    point_x_y_list.append((x, y))  # 将本次生成的数据存储到列表中
    x = y  # 将x的值改为此次y的值
print(point_x_y_list)

[(0, 1), (1, 3), (3, 7), (7, 15), (15, 31)]


小总结：
>**生成数据这个功能已经实现了，但是没有解决 生成的的数据个数不确定的问题**

### 1.2.2. <a id='toc1_2_2_'></a>[尝试2](#toc0_)
上面的问题中 提到需要的个数不确定，那么此时我们可以考虑使用”迭代器“，参考代码如下：

In [2]:
class PointXY(object):
    """
    通过迭代器，生成不确定个数的点的坐标
    """
    def __init__(self):
        self.x = 0

    def __iter__(self):
        return self

    def __next__(self):
        temp_y = 2 * self.x + 1
        temp_point_x_y = (self.x, temp_y)  # 得到一个元组
        self.x = temp_y
        return temp_point_x_y


point_x_y = PointXY()  # 创建一个可迭代对象
point_x_y_iter = iter(point_x_y)  # 获取迭代器

point_x_y_1 = next(point_x_y_iter)
print(point_x_y_1)

point_x_y_2 = next(point_x_y_iter)
print(point_x_y_2)

point_x_y_3 = next(point_x_y_iter)
print(point_x_y_3)

point_x_y_4 = next(point_x_y_iter)
print(point_x_y_4)


(0, 1)
(1, 3)
(3, 7)
(7, 15)


小总结：

>**实现了生成不确定个数的问题，但是 如果想要在第X次，例如第5次时，修改y=2x+1中的2或者1的值，则不能满足**

想要解决这个问题其实很简单，下面先学习生成器，然后你自然就知道怎样解决这个问题

## 1.3. <a id='toc1_3_'></a>[生成器](#toc0_)
### 1.3.1. <a id='toc1_3_1_'></a>[什么是生成器](#toc0_)
通过列表生成式，我们可以直接创建一个列表。但是，受到内存限制，列表容量肯定是有限的。而且，创建一个包含100万个元素的列表，不仅占用很大的存储空间，如果我们仅仅需要访问前面几个元素，那后面绝大多数元素占用的空间都白白浪费了。

所以，如果列表元素可以按照某种算法推算出来，那我们是否可以在循环的过程中不断推算出后续的元素呢？这样就不必创建完整的list，从而节省大量的空间。

通俗的理解：

在Python中，这种一边循环一边计算的机制，称为生成器：generator

### 1.3.2. <a id='toc1_3_2_'></a>[生成器与迭代器](#toc0_)
其实利用迭代器，我们也可以在每次迭代获取数据（通过next()方法）时按照特定的规律进行生成。

但是在实现一个迭代器时，关于当前迭代到的状态需要我们自己记录，进而才能根据当前状态生成下一个数据。

为了达到记录当前状态，并配合next()函数进行迭代使用，

简言之，迭代器是可以实现在循环的过程中生成数据的，但是稍稍有些复杂

有一个能具有迭代器的功能，且比它更加简单的方式：生成器(generator)

生成器是一类特殊的迭代器

### 1.3.3. <a id='toc1_3_3_'></a>[创建生成器方法1](#toc0_)
要创建一个生成器，有多种方法

第一种方法很简单，只要把一个列表生成式的 [ ] 改成 ( )

In [5]:
nums = list(range(5)) #* [x for x in range(5)]
print(type(nums))
print(nums)

nums2 = iter(range(5)) #* (x for x in range(5))
print(type(nums2))
print(nums2)

<class 'list'>
[0, 1, 2, 3, 4]
<class 'generator'>
<generator object <genexpr> at 0x000001AE5A77E9E0>


说明：

1. 区别仅在于最外层的 [ ] 和 ( ) ， nums是一个列表，而 nums2是一个生成器

2. 可以直接打印出列表nums的每一个元素，而对于生成器nums2，我们可以按照迭代器的使用方法来使用，即可以通过next()函数、for循环、list()等方法使用

In [7]:
nums = list(range(5)) #* [x for x in range(5)]
print(type(nums))
print(nums)

nums2 = iter(range(5)) #* (x for x in range(5))
print(type(nums2))
print(nums2)

print(next(nums2))

print(next(nums2))

print(next(nums2))

print(next(nums2))

print(next(nums2))

print(next(nums2)) #! 既然生成器是特殊的一种迭代器，那么生成器也会异常

<class 'list'>
[0, 1, 2, 3, 4]
<class 'range_iterator'>
0
1
2
3
4


StopIteration: 

### 1.3.4. <a id='toc1_3_4_'></a>[创建生成器方法2](#toc0_)

generator功能非常强大

如果推算的算法比较复杂，用类似列表生成式的 for 循环无法实现的时候，还可以用之前学习的函数来自己实现一个生成器

下面以斐波那契数列来举例

>斐波那契数列指的是这样一个数列 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233，377，610，987，1597，2584，4181，6765，10946，17711，28657，46368........
>这个数列从第3项开始，每一项都等于前两项之和。

如果想要知道怎样得到这样的数列，简单代码如下：

In [8]:
num1 = 1
num2 = 1

print("第1个值", num1)

num1, num2 = num2, num1+num2
print("第2个值", num1)

num1, num2 = num2, num1+num2
print("第3个值", num1)

num1, num2 = num2, num1+num2
print("第4个值", num1)

第1个值 1
第2个值 1
第3个值 2
第4个值 3


下面使用迭代器的方式来实现一个不确定个数的斐波那契数列，参考代码如下：

In [9]:
class FibIterator(object):
    """斐波那契数列迭代器"""
    def __init__(self):
        # num1用来保存前前一个数，初始值为数列中的第一个数1
        self.num1 = 1
        # num2用来保存前一个数，初始值为数列中的第二个数1
        self.num2 = 1

    def __next__(self):
        """被next()函数调用来获取下一个数"""
        temp_num = self.num1
        self.num1, self.num2 = self.num2, self.num1+self.num2
        return temp_num

    def __iter__(self):
        """迭代器的__iter__返回自身即可"""
        return self


fib = FibIterator()

# 因为fib是迭代器所以不必使用iter()函数，直接使用next()函数即可
print(next(fib))
print(next(fib))
print(next(fib))
print(next(fib))
print(next(fib))
print(next(fib))


1
1
2
3
5
8


可以看到上面的代码，虽然能够实现功能，但是稍稍有些复杂

是否有更加简单的方式呢？

看如下代码，使用生成器实现的斐波那契数列

代码1（不是生成器，不能实现斐波那契数列）

In [10]:
def fib_generator():
    num1 = 1
    num2 = 1
    while True:
        temp_num = num1
        num1, num2 = num2, num1+num2
        return temp_num  # 方式1代码
        # yield temp_num

# 方式1代码（方式1不能够生成1，1，2，3，5...斐波那契数列）
print(fib_generator())
print(fib_generator())
print(fib_generator())
print(fib_generator())

# 方式2代码（可以生成斐波那契数列）
# fib = fib_generator()
# print(next(fib))
# print(next(fib))
# print(next(fib))
# print(next(fib))


1
1
1
1


代码2（是生成器，能实现斐波那契数列）

In [11]:
def fib_generator():
    num1 = 1
    num2 = 1
    while True:
        temp_num = num1
        num1, num2 = num2, num1+num2
        # return temp_num  # 方式1代码
        yield temp_num

# 方式1代码（方式1不能够生成1，1，2，3，5...斐波那契数列）
# print(fib_generator())
# print(fib_generator())
# print(fib_generator())
# print(fib_generator())

# 方式2代码（可以生成斐波那契数列）
fib = fib_generator()
print(next(fib))
print(next(fib))
print(next(fib))
print(next(fib))


1
1
2
3


在使用生成器实现的方式中，我们将原本在迭代器__next__方法中实现的基本逻辑放到一个函数中来实现，但是将每次迭代返回数值的return换成了yield，此时新定义的函数便不再是函数，而是一个生成器了

简单来说：只要在def函数中有yield关键字的 就称为 生成器

此时按照调用函数的方式( fib = fib_generator() )就不再是执行函数体了，而是会返回一个生成器对象，然后就可以按照使用迭代器(因为生成器是一种特殊的迭代器)的方式来使用生成器了

### 1.3.5. <a id='toc1_3_5_'></a>[yield关键字](#toc0_)

上面的代码，在函数中使用了yield关键字，替换了return，虽然看上去仅仅是关键字不同，但是功能效果已经决然不同了

说明
1. 只要有yield关键字，那么虽然看上去是调用函数，实际上已经变成了创建一个 生成器对象
2. 通过next调用 生成器，可以让 这个带有yield的def代码块，开始执行
    1. 如果是第一次执行，则从def代码块的开始部分执行，直到遇到yield为止，并且把yield关键字后的数值返回，当做next()的返回值
    2. 如果不是第一次执行，则从上一次暂停的位置执行(即从上一次yield关键字的下一个语句开始执行)，直到遇到下一次yield为止，并且把yield关键字后的数值返回，当做next()的返回值</br>
   
更好的理解带有yield关键字的函数，参考代码如下

In [12]:
def fib_generator():
    print("---1---")
    num1 = 1
    num2 = 1
    while True:
        print("---2---")
        temp_num = num1
        print("---3---")
        num1, num2 = num2, num1+num2
        print("---4---")
        yield temp_num
        print("---5---")



fib = fib_generator()

print("执行第1次的next函数---开始--")
num = next(fib)
print("执行第1次的next函数---结束--")

print("打印第1次next返回的结果--开始--")
print(num)
print("打印第1次next返回的结果--结束--")

print("执行第2次的next函数---开始--")
num = next(fib)
print("执行第2次的next函数---结束--")


print("打印第2次next返回的结果--开始--")
print(num)
print("打印第2次next返回的结果--结束--")


执行第1次的next函数---开始--
---1---
---2---
---3---
---4---
执行第1次的next函数---结束--
打印第1次next返回的结果--开始--
1
打印第1次next返回的结果--结束--
执行第2次的next函数---开始--
---5---
---2---
---3---
---4---
执行第2次的next函数---结束--
打印第2次next返回的结果--开始--
1
打印第2次next返回的结果--结束--


注意：

1. 要知道一个def代码块，只要有yield就不再是函数，而是生成器
2. 要知道调用def代码块(普通函数)与调用带有yield的def代码块(生成器)的调用方式不同
3. 要注意return与yield的功能也不同
    1. return接收一个函数，且有返回值
    2. yield暂停执行一个函数，且有返回值
4. 面试中也会经常用到，一定要将yield掌握

### 1.3.6. <a id='toc1_3_6_'></a>[获取生成器def代码块中return值](#toc0_)
测试代码1：

In [13]:
def fib_generator():
    num1 = 1
    num2 = 1
    temp_num = num1
    num1, num2 = num2, num1+num2
    yield temp_num

    temp_num = num1
    num1, num2 = num2, num1+num2
    yield temp_num

    temp_num = num1
    num1, num2 = num2, num1+num2
    yield temp_num

    return "已经生成了3个斐波那契数列的值..."


fib = fib_generator()

print(next(fib))
print(next(fib))
print(next(fib))
# print(next(fib))

1
1
2


测试代码2：

In [14]:
def fib_generator():
    num1 = 1
    num2 = 1
    temp_num = num1
    num1, num2 = num2, num1+num2
    yield temp_num

    temp_num = num1
    num1, num2 = num2, num1+num2
    yield temp_num

    temp_num = num1
    num1, num2 = num2, num1+num2
    yield temp_num

    return "已经生成了3个斐波那契数列的值..."


fib = fib_generator()

print(next(fib))
print(next(fib))
print(next(fib))
print(next(fib))

1
1
2


StopIteration: 已经生成了3个斐波那契数列的值...

测试代码3：

In [15]:
def fib_generator():
    num1 = 1
    num2 = 1
    temp_num = num1
    num1, num2 = num2, num1+num2
    yield temp_num

    temp_num = num1
    num1, num2 = num2, num1+num2
    yield temp_num

    temp_num = num1
    num1, num2 = num2, num1+num2
    yield temp_num

    return "已经生成了3个斐波那契数列的值..."


fib = fib_generator()

print(next(fib))
print(next(fib))
print(next(fib))

try:
    print(next(fib))
except StopIteration as ret:
    print(ret.value)


1
1
2
已经生成了3个斐波那契数列的值...


小总结：

>调用generatr时，发现拿不到generator的return语句的返回值。如果想要拿到返回值，必须捕获StopIteration错误，返回值包含在StopIteration的value中

## 1.4. <a id='toc1_4_'></a>[使用send唤醒](#toc0_)
我们除了可以使用next()函数来唤醒，让生成器继续执行外，还可以使用send()函数来唤醒执行。

使用send()函数的一个好处是：可以在唤醒的同时向断点处传入一个附加数据

### 1.4.1. <a id='toc1_4_1_'></a>[测试send](#toc0_)
看如下测试代码：

In [16]:
def generator_test():
    while True:
        print("--1--")
        num = yield 100
        print("--2--", "num=", num)


g = generator_test()

print(next(g))
print(next(g))
print(next(g))


--1--
100
--2-- num= None
--1--
100
--2-- num= None
--1--
100


使用send修改上面的代码，示例如下：

In [22]:
def generator_test():
    while True:
        print("--1--")
        num = yield 100
        print("--2--", "num=", num)


g = generator_test()

# 第一次调用，会将yield 100执行完毕后暂停代码执行，注意此时 并没有给num值
print(next(g))
# print(next(g))
# print(next(g))

# send会让生成器从上次停止的位置 继续开始执行，并且会将11传递到生成器中
# 当做上一次执行yield 100 这个表达式的结果
# 然后在第4行代码，就可以想象成 num = 11了，因为100代表上次yield 100的结果
# 直到遇到下一次的yield 暂停运行，并且把100返回，此时就也打印了100
print(g.send(11))

# 与上次send(11)类似，只不过此次将22当做给yield 100这个表示的结果 给num
print(g.send(22))


--1--
100
--2-- num= 11
--1--
100
--2-- num= 22
--1--
100


### 1.4.2. <a id='toc1_4_2_'></a>[注意](#toc0_)
测试代码：

In [21]:
def generator_test():
    while True:
        print("--1--")
        num = yield 100
        print("--2--", "num=", num)


g = generator_test()

# print(g.send(None))  # 可以将这行屏蔽，看看不同
print(g.send(11))

TypeError: can't send non-None value to a just-started generator

## 1.5. <a id='toc1_5_'></a>[总结](#toc0_)
1. 使用了yield关键字的函数不再是函数，而是生成器

2. yield关键字有两点作用：
    1. 保存当前运行状态（断点），然后暂停执行，即将生成器（函数）挂起
    2. 将yield关键字后面表达式的值作为返回值返回，此时可以理解为起到了return的作用
3. 可以使用next()函数让生成器从断点处继续执行，即唤醒生成器（函数）

4. Python3中的生成器可以使用return返回最终运行的返回值

5. 生成器是这样一个函数，它记住上一次返回时在函数体中的位置。对生成器函数的第二次（或第 n 次）调用跳转至该函数中间，而上次调用的所有局部变量都保持不变。生成器不仅“记住”了它数据状态；生成器还“记住”了它在流控制构造（在命令式编程中，这种构造不只是数据值）中的位置。

6. 生成器的特点：

    1. 存储的是生成数据的方式（即算法），而不是存储生成的数据，因此节约内存

## 1.6. <a id='toc1_6_'></a>[作业](#toc0_)
既然生成器已经学习完毕了，想想该怎样完成 在本文最开始时提到的那个问题呢？

参考代码：

In [33]:
def create_point():
    x=0
    y=2*x+1

    while True:
        num=yield x,y
        x,y = (y, 2*y+1) if num is None else (y, num[0]*y+num[1])
         
            
test=create_point()

print(next(test))
print(next(test))
print(next(test))
print(next(test))
print(test.send((3,2)))

(0, 1)
(1, 3)
(3, 7)
(7, 15)
(15, 47)
